# Sequence to Sequence model for english to tamil transilation using rnn encoder and decoder ( at character level)

## dataset is downloaded from http://www.manythings.org/anki/

In [80]:
from __future__ import print_function

from keras.models import Model
from keras.layers import Input,LSTM,Dense
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=64
epochs=25
latent_dim=256
data_path="/home/santhosh/keras/rnn/rnn_encoder_decoder/data/tam.txt"

## vectorizing the data

In [81]:
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,'r',encoding='utf-8') as f:
    lines=f.read().split('\n')
for line in lines:
    line=line.split('\t')
    if(len(line)!=2):
        continue
    input_text,target_text=line
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text='\t'+target_text+'\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [55]:
input_characters=sorted(list(input_characters))
targer_characters=sorted(list(target_characters))
num_encoder_tokens=len(input_characters)
num_decoder_tokens=len(target_characters)
max_encoder_seq_len=max([len(text) for text in input_texts])
max_decoder_seq_len=max([len(text) for text in target_texts])

print('Number of samples:',len(input_texts))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


Number of samples: 180
number of unique input token: 53
number of unique output token: 54
Max Sequence length for inputs: 94
Max Sequence length for outputs: 111


## defining token2index

In [82]:
input_token2index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token2index=dict([(char,i) for i,char in enumerate(target_characters)])

## defing encoder_input,decoder_input and decoder_output

In [83]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_input_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_texts),max_decoder_seq_len,num_decoder_tokens),dtype='float32')


## creating training dataset

In [84]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
    for t,char in enumerate(input_text):
        encoder_input_data[i,t,input_token2index[char]]=1
    for t,char in enumerate(target_text):
        decoder_input_data[i,t,target_token2index[char]]=1
        # decoder_target_data is ahead of decoder_input_data by one timestep
        if t>0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i,t-1,target_token2index[char]]=1


## Define an input sequence and process it.

In [85]:

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]


## Set up the decoder, using encoder_states as initial state.

In [86]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

## Define the model that will turn
## `encoder_input_data` & `decoder_input_data` into `decoder_target_data`

In [87]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])

## function to test model

In [88]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token2index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token2index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token2index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_len):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


## Run training

In [ ]:

iteration=0

# load weights
print('loading the weights')
model.load_weights("weights_best.hdf5")

# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))


iteration_file="/home/santhosh/keras/rnn/rnn_encoder_decoder/iteration.txt"
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    
except:
    print('no file exist')

# checkpoint
filepath="weights_best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    print('Iteration:',iteration+1)
    #training
    model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)
    #prepare sample_data to test 5 samples:
    print("-"*50)
    for i in range(5):
        index=np.random.randint(len(input_texts))
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=decode_sequence(encoded_input_sequence)
        print(input_texts[index],output_sequence)
    print("-"*50)
    # Save model
    file=open(iteration_file,'a')
    file.write('iteration:'+str(iteration+1)+'\n')
    file.close()
    iteration+=1
    model.save('s2s.h5')

loading the weights
acc: 22.12%


file_data, iteration:4
Iteration: 5
Train on 144 samples, validate on 36 samples
Epoch 1/25
144/144 [==============================] - 6s 42ms/step - loss: 0.0770 - acc: 0.2392 - val_loss: 1.7445 - val_acc: 0.1374


/home/santhosh/anaconda3/envs/pydl/lib/python3.5/site-packages/keras/engine/network.py:888: UserWarning: Layer lstm_7 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_6/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_6/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


Epoch 2/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0638 - acc: 0.2419 - val_loss: 1.6927 - val_acc: 0.1426
Epoch 3/25
144/144 [==============================] - 3s 21ms/step - loss: 0.0569 - acc: 0.2423 - val_loss: 1.6801 - val_acc: 0.1441
Epoch 4/25
144/144 [==============================] - 3s 21ms/step - loss: 0.0522 - acc: 0.2437 - val_loss: 1.7053 - val_acc: 0.1399
Epoch 5/25
144/144 [==============================] - 3s 22ms/step - loss: 0.0519 - acc: 0.2432 - val_loss: 1.7104 - val_acc: 0.1391
Epoch 6/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0530 - acc: 0.2432 - val_loss: 1.7075 - val_acc: 0.1411
Epoch 7/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0513 - acc: 0.2435 - val_loss: 1.7165 - val_acc: 0.1376
Epoch 8/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0537 - acc: 0.2419 - val_loss: 1.7168 - val_acc: 0.1426
Epoch 9/25
144/144 [==============================] - 3s 20ms/step - 

144/144 [==============================] - 3s 20ms/step - loss: 0.0454 - acc: 0.2459 - val_loss: 1.8177 - val_acc: 0.1381
Epoch 7/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0455 - acc: 0.2451 - val_loss: 1.8007 - val_acc: 0.1364
Epoch 8/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0458 - acc: 0.2456 - val_loss: 1.8091 - val_acc: 0.1379
Epoch 9/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0459 - acc: 0.2456 - val_loss: 1.8260 - val_acc: 0.1359
Epoch 10/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0458 - acc: 0.2449 - val_loss: 1.8162 - val_acc: 0.1371
Epoch 11/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0460 - acc: 0.2461 - val_loss: 1.8177 - val_acc: 0.1354
Epoch 12/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0472 - acc: 0.2451 - val_loss: 1.8127 - val_acc: 0.1404
Epoch 13/25
144/144 [==============================] - 3s 19ms/step - loss: 0

144/144 [==============================] - 3s 20ms/step - loss: 0.0433 - acc: 0.2466 - val_loss: 1.8816 - val_acc: 0.1336
Epoch 12/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0438 - acc: 0.2456 - val_loss: 1.8558 - val_acc: 0.1359
Epoch 13/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0435 - acc: 0.2450 - val_loss: 1.8606 - val_acc: 0.1351
Epoch 14/25
144/144 [==============================] - 3s 21ms/step - loss: 0.0428 - acc: 0.2462 - val_loss: 1.8457 - val_acc: 0.1371
Epoch 15/25
144/144 [==============================] - 3s 21ms/step - loss: 0.0433 - acc: 0.2467 - val_loss: 1.8779 - val_acc: 0.1266
Epoch 16/25
144/144 [==============================] - 3s 19ms/step - loss: 0.1855 - acc: 0.2066 - val_loss: 1.7139 - val_acc: 0.1326
Epoch 17/25
144/144 [==============================] - 3s 20ms/step - loss: 0.1196 - acc: 0.2251 - val_loss: 1.7382 - val_acc: 0.1364
Epoch 18/25
144/144 [==============================] - 3s 19ms/step - loss

144/144 [==============================] - 3s 19ms/step - loss: 0.0464 - acc: 0.2462 - val_loss: 1.8301 - val_acc: 0.1349
Epoch 16/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0454 - acc: 0.2453 - val_loss: 1.8258 - val_acc: 0.1364
Epoch 17/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0456 - acc: 0.2445 - val_loss: 1.8284 - val_acc: 0.1349
Epoch 18/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0457 - acc: 0.2452 - val_loss: 1.8395 - val_acc: 0.1329
Epoch 19/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0452 - acc: 0.2451 - val_loss: 1.8297 - val_acc: 0.1336
Epoch 20/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0452 - acc: 0.2456 - val_loss: 1.8333 - val_acc: 0.1349
Epoch 21/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0455 - acc: 0.2446 - val_loss: 1.8444 - val_acc: 0.1319
Epoch 22/25
144/144 [==============================] - 3s 19ms/step - loss

Epoch 20/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0437 - acc: 0.2458 - val_loss: 1.8585 - val_acc: 0.1391
Epoch 21/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0440 - acc: 0.2459 - val_loss: 1.8563 - val_acc: 0.1389
Epoch 22/25
144/144 [==============================] - 3s 20ms/step - loss: 0.0440 - acc: 0.2450 - val_loss: 1.8664 - val_acc: 0.1326
Epoch 23/25
144/144 [==============================] - 3s 19ms/step - loss: 0.0449 - acc: 0.2449 - val_loss: 1.8732 - val_acc: 0.1349
Epoch 24/25
 64/144 [============>.................] - ETA: 1s - loss: 0.0414 - acc: 0.2514